# Simulation 02 - BathseLock-NS-High

#### Import libraries

In [1]:
# package(s) related to time, space and id
import datetime
import platform
import random
import os

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# tranport network analysis package
import transport_network_analysis.core as core
import transport_network_analysis.graph_module as graph_module

# Used for mathematical functions
import math             

# Used for making the graph to visualize our problem
import networkx as nx 

# Random and distribution for simulating
import random

# Create the simpy environment
env = simpy.Environment()

### Define the graph

In [2]:
# Graph location
location_graph = "Shape-Files/Rotterdam-Antwerpen-corridor"
name_graph = "edges_2.shp"

##  create a graph of the loaded data, 
FG = nx.read_shp(os.path.join(location_graph, name_graph), simplify=True)
graph = graph_module.Graph()

lat_lon_to_index = {}
edge_id_counter = 0

HEIGHT_RESTRICED_NODES = {'77': 9.2, '183':8.7}
CLOSED_NODES = [365, 73, 208]
MOVABLE_BRIDGES = [340, 103]

# Define locks
Lock = type('Lock',(core.Identifiable, core.Log, core.IsLock, core.HasResource), {})

LOCKS = [
    Lock(**{'env': env, 'name':"Krammerlock",'nodes':[178,179], 'neighbour_lock':[317, 365],'lock_length':280,'lock_width':24, 'doors_open': 60, 'doors_close':60, 'operating_time':1440, 'nr_resources':3, 'priority': True}), 
    Lock(**{'env': env, 'name':"Hansweertlock",'nodes':[220,221], 'neighbour_lock':[208, 209], 'lock_length':280,'lock_width':24, 'doors_open': 60, 'doors_close':60, 'operating_time':720, 'nr_resources':3, 'priority': True}), 
    Lock(**{'env': env, 'name':"Kreekraklock",'nodes':[155, 156], 'neighbour_lock':[73,72], 'lock_length':320,'lock_width':24, 'doors_open': 60, 'doors_close':60, 'operating_time':840, 'nr_resources':3, 'priority': True}),
    Lock(**{'env': env, 'name':"Krammerlock",'nodes':[317, 365], 'neighbour_lock':[178,179],'lock_length':280,'lock_width':24, 'doors_open': 60, 'doors_close':60, 'operating_time':1440, 'nr_resources':3, 'priority': True}), 
    Lock(**{'env': env, 'name':"Hansweertlock",'nodes':[208, 209], 'neighbour_lock':[220,221], 'lock_length':280,'lock_width':24, 'doors_open': 60, 'doors_close':60, 'operating_time':720, 'nr_resources':3, 'priority': True}), 
    Lock(**{'env': env, 'name':"Kreekraklock",'nodes':[73,72], 'neighbour_lock':[155, 156], 'lock_length':320,'lock_width':24, 'doors_open': 60, 'doors_close':60, 'operating_time':840, 'nr_resources':3, 'priority': True})
]

AttributeError: module 'transport_network_analysis.core' has no attribute 'IsLock'

In [ ]:
# calculate distance between two points
def calculate_distance(orig, dest):
    wgs84 = pyproj.Geod(ellps='WGS84')
    distance = wgs84.inv(shapely.geometry.asShape(orig).x, shapely.geometry.asShape(orig).y, 
                         shapely.geometry.asShape(dest).x, shapely.geometry.asShape(dest).y)[2]
    return distance


def get_lock(node_one, node_two):
    for lock in LOCKS:
        nodes_lock = lock.nodes
        if node_one in nodes_lock and node_two in nodes_lock:
            return lock
    return None

#new
def get_bridge(index):
    if index in MOVABLE_BRIDGES:
        return True
    return False


def get_slow_down(index):
    if (get_bridge(index)):
        return True
    for lock in LOCKS:
        if index in lock.nodes:
            return True
    return False
    

def get_height_restriction(index):
    '''
    Restriction in meters for a node, vessel can only pass if height < restriction
    '''
    if (str(index) in HEIGHT_RESTRICED_NODES.keys()):
        print("found restriction {}".format(HEIGHT_RESTRICED_NODES[str(index)]))
        return HEIGHT_RESTRICED_NODES[str(index)]
    return 99

def get_closed(index):
    if index in CLOSED_NODES:
        return True
    return False

          
def do_nodes():
    for i, node in enumerate(FG.nodes(data = True)):
        graph.graph.add_node(i, pos = node[0], 
                        height_restriction = get_height_restriction(i),
                        closed = get_closed(i),
                        name = "Node {}".format(i),
                        bridge = get_bridge(i),
                        slow_down = get_slow_down(i),
                        geometry = shapely.geometry.Point(node[0][0], node[0][1]))

        lat_lon_to_index[node[0]] = i
        

def do_edges():
    for edge in FG.edges(data=True):
        edge_type = 'Canal'
        lock = get_lock(lat_lon_to_index[edge[0]],lat_lon_to_index[edge[1]])
        edge_attribute = None
        resource = simpy.Resource(env, capacity = math.inf)
        if (lock != None):
            edge_type = 'Lock'
            edge_attribute = lock
            resource = lock.resource
            print(lock.name)
        graph.graph.add_edge(lat_lon_to_index[edge[0]], 
                     lat_lon_to_index[edge[1]],
                     Object=edge_type,
                     dis= calculate_distance(nx.get_node_attributes(graph.graph, 'geometry')[lat_lon_to_index[edge[1]]], 
                                             nx.get_node_attributes(graph.graph, 'geometry')[lat_lon_to_index[edge[0]]]),
                     weight = 1,
                     Resources = resource,
                     attribute = edge_attribute
                    )

        
do_nodes()
do_edges()

FG = graph.graph
FG_S = graph.graph.to_directed()

In [ ]:
## Grapgh obtained from FG from build graph

def path_length(some_path):
    wgs84 = pyproj.Geod(ellps='WGS84')
    some_path_dist = 0
    for node in enumerate(some_path):
        orig = nx.get_node_attributes(FG, "geometry")[some_path[node[0]]]
        dest = nx.get_node_attributes(FG, "geometry")[some_path[node[0] + 1]]
        some_path_dist += wgs84.inv(shapely.geometry.asShape(orig).x, shapely.geometry.asShape(orig).y, 
                                         shapely.geometry.asShape(dest).x, shapely.geometry.asShape(dest).y)[2]
        if node[0] + 2 == len(some_path):
                    break
    return some_path_dist

def get_restrictions(path, graph):
    """
    Pickup all restrictions for a given path
    Restrictions are put on the nodes when creating the graph. 
    Now loop over the nodes and check if they have a restriction.
    Restrictions like height or closed nodes but others are possible too.
    """
    restrictions = {'closed':False, 'height': 99}
    height_restriction_on_graph = nx.get_node_attributes(graph, 'height_restriction')
    closed_nodes = nx.get_node_attributes(graph, 'closed')
    for node in path:
        rest = height_restriction_on_graph[node]
        if rest < restrictions['height']:
            restrictions['height'] = rest
        if closed_nodes[node]:
            restrictions['closed'] = True
    return restrictions    
    
class NodePath():
    """
    This object contains some more info about the path
    like: restrictions, length in m etc. 
    Can be used later when selecting proper path for a vessel.
    """
    def __init__(self, path, graph, indentifier):
        self.nodes = path
        self.length = path_length(path)
        self.name = "Path_{}".format(indentifier)
        # get restrictions for the nodes
        restrictions = get_restrictions(path, graph)
        self.max_height = restrictions['height']
        self.closed = restrictions['closed']
        
    def __type__(self):
        return "node_path" 
    
    def __str__(self):
        return "from node {} to  node {}, covering distance: {}, closed:{}, max height: {}".format(self.nodes[-1], self.nodes[0] ,self.length, self.closed, self.max_height)
    def plot(self, graph, save=False, filename=''):
        """
        Plot the path
        """
        locks = [178, 179, 220, 221, 155, 156]
        #new
        bridges = [340,103]
        for key in dict(graph.nodes):
            graph.nodes[key]["Path"] = 0.5
            graph.nodes[key]["Color"] = "Red"
            for i, node in enumerate(self.nodes):
                if key == node:
                    if i == 0:
                        graph.nodes[key]["Path"] = 200
                        graph.nodes[key]["Color"] = "Yellow"
                    elif i == len(self.nodes) - 1:
                        graph.nodes[key]["Path"] = 200
                        graph.nodes[key]["Color"] = "Blue"
                    elif node in locks:
                        # here we give other color to the locks
                        graph.nodes[key]["Path"] = 200
                        graph.nodes[key]["Color"] = "Red"
                    elif node in bridges:
                        graph.nodes[key]["Path"] = 200
                        graph.nodes[key]["Color"] = "Orange"
                    else:
                        graph.nodes[key]["Path"] = 50
                        graph.nodes[key]["Color"] = "Green"
                        break

        plt.figure(figsize=(18,18))
        nx.draw(graph, nx.get_node_attributes(graph, 'pos'), 
                with_labels = True, 
                node_size = list(dict(nx.get_node_attributes(graph, 'Path')).values()), 
                font_size = 3, 
                node_color = list(dict(nx.get_node_attributes(graph, 'Color')).values()), 
                width = 0.4, 
                arrowsize = 10)
        if (save):
            plt.savefig('img/' + filename + '.png')
        plt.show()

# Get all simple paths (which are lists of nodes)
# Take the list of nodes and put it in a NodePath object as shown above
# This object contains some more info about the path
# like: restrictions, length in m etc. 

paths = []
i = 1
for path in nx.all_simple_paths(FG_S, 263, 41): # simplepaths geeft lijst met node nummers
    #paths.append(NodePath(path, FG_S, i)) # maak van ieder lijst met nodes een NodePath obect en stop het in lijst
    #i+=1
    if i == 2:
        paths.append(NodePath(path, FG_S, "Scheldt-Rhine"))
        
    elif i == 15:
        paths.append(NodePath(path, FG_S, "Zuid-Beveland"))
     #new for bathseroute !!!  
    elif i == 5:
        paths.append(NodePath(path, FG_S, "Bathse-canal"))
        
    else: paths.append(NodePath(path, FG_S, i)) # maak van ieder lijst met nodes een NodePath obect en stop het in lijst
    i+=1
        
    
# this returns al the node paths sorted on length (shortest path first)
ordered_paths = sorted(paths, key=lambda paths: paths.length)

## Create Vessels from DB

In [ ]:
vessel_db = pd.read_csv("Results/Vessels.csv")
vessel_db.head()

In [ ]:
def init_random_vessel(routes, vessel_database, number):
    """Initialize a random vessel - parameters obtained from database"""
    
    vessel_data = {"env": None}
    
    TransportResource = type('Ship', 
                             (core.Identifiable, core.Log, core.ContainerDependentMovable, 
                              core.HasEnergy, core.HasRestrictions,
                              core.HasResource, core.Routeable), 
                             {})
    
    # Select random vessel from database
    vessel_info = vessel_database.sample(n = 1)
    vessel_data["name"] = "Vessel " + str(number)
    vessel_data["id"] = vessel_info["VesselID"].values[0]
    
    
    # Select a random route - taking restrictions in mind
    correction = 0
    
    if random.random() < 0.35:
        correction = 8.7
    
    vessel_data["loaded"] = vessel_info["is_loaded"].values[0]
    vessel_data["length"] = vessel_info["length"].values[0]
    vessel_data["width"] = vessel_info["width"].values[0]
    
    for path in routes:
        if path.closed:
            continue
        
        if vessel_data["loaded"]:
            vessel_data["height"] = vessel_info["height_loaded"].values[0]
            vessel_data["resistance"] = vessel_info["loaded_resistance"].values[0]
            speed = vessel_info["loaded_speed"].values[0]
            
            if vessel_data["height"] >= path.max_height or correction >= path.max_height:
                continue
        
        else:
            vessel_data["height"] = vessel_info["height"].values[0]
            vessel_data["resistance"] = vessel_info["resistance_unloaded"].values[0]
            speed = vessel_info["speed_unloaded"].values[0]
            
            if vessel_data["height"] >= path.max_height or correction >= path.max_height:
                continue
        
        break
    
    # 50/50 choses different direction
    if random.random() < 0.5:
        vessel_data["route"] = path.nodes[::-1]
        vessel_data["complete_path"] = path
    else:
        vessel_data["route"] = path.nodes
        vessel_data["complete_path"] = path
    
    vessel_data["geometry"] = vessel_data["route"][0]
    
    # Initialize remaining vessel properties
    vessel_data["length"] = vessel_info["length"].values[0]
    
    def compute_v_provider(v_empty, v_full):
        return lambda x: (v_empty + v_full) / 2
    
    vessel_data["compute_v"] = compute_v_provider(speed, speed)
    vessel_data["emissionfactor"] = vessel_info["emissionfactor"].values[0]
    vessel_data["capacity"] = vessel_info["loaded_capacity"].values[0]
    
    return TransportResource(**vessel_data)

### Run simulation

In [ ]:
def start(env, vessel):
    while True:
        
        vessel.log_entry("Start sailing", env.now, "0", vessel.geometry)
        yield from vessel.move()
        vessel.log_entry("Stop sailing", env.now, "0", vessel.geometry)

        if vessel.geometry == nx.get_node_attributes(FG, "geometry")[vessel.route[-1]]:
            break

In [ ]:
%%time

# Add graph to environment
env.FG = FG

env.paths = {}

# 24hours
SIM_TIME = 60 * 60  * 24 * 31
vessels = []

def pois(t):
    hour = int((t%(24*3600))/3600)
    average_on_hour = [5,6,7,8,10,11,12,16,18,19,19,20,19,18,17,15,15,13,11,10,9,7,6,4]
    number_of_boats_this_hour = np.random.normal(average_on_hour[hour])
    if number_of_boats_this_hour == 0:
        return 0
    return 60 * 60 / number_of_boats_this_hour

def setup(env, distribution):
    # initial vessel
    i = 1
    vessel = init_random_vessel(ordered_paths, vessel_db, i)
    vessel.env = env
    vessel.geometry = nx.get_node_attributes(FG, "geometry")[vessel.route[0]]
    env.process(start(env, vessel))
    vessels.append(vessel)

    while True:
        #break
        # timeout before next ship is released
        i += 1
        yield env.timeout(distribution(env.now))
        vessel = init_random_vessel(ordered_paths, vessel_db, i)
        vessel.env = env
        vessel.geometry = nx.get_node_attributes(FG, "geometry")[vessel.route[0]]
        env.process(start(env, vessel))
        vessels.append(vessel)
         
env.process(setup(env, pois))

env.crossover_points = {182: ((184, 394,154,155,156,216,217, 218), (164, 134,133, 132,73,72,71,402,401)),
                        2: ((218,217,216,156,155,154,394,184), (401, 402,71,72,73,132,133,134,164)),
                        141: ((140,139,179,178,379), (363, 364, 365, 317, 403)),
                        382: ((379,178,179,139,140), (403,317,365,364,363)),
                        273:((222,221,220,279), (272, 209, 208, 210)),
                        278: ((279,220,221,222), (210,208, 209,272))                        
}

env.run(until = SIM_TIME)

## Save all results

In [ ]:
# Save location for files
save_location = "Results/"

In [ ]:
# Run to get a unique id for a new simulation
import uuid
str(uuid.uuid1())

In [ ]:
# to add information to dataframes
def append_dataframe(save_location, object_id, overwrite = False):
    """
    Check if dataframe is alfready filled with information, if not append.
    If it is filled with similar values, raise an error unless self.overwrite == True.
    """
    
    try:
        existing_df = pd.read_csv(save_location + "simulations.csv")
    except FileNotFoundError:
        existing_df = pd.DataFrame.from_dict({"SimulationID": [], "SimulationName": []})

    if object_id.id not in list(existing_df["SimulationID"]):
        existing_df = existing_df.append({"SimulationID": object_id.id, "SimulationName": object_id.name}, ignore_index=True)
        
    elif overwrite == True:
        existing_df = existing_df[existing_df["SimulationID"] != object_id.id]
        existing_df = existing_df.append({"SimulationID": object_id.id, "SimulationName": object_id.name}, ignore_index=True)
        
    else:
        raise KeyError("Simulation ID or simulation name already exist. " + 
                        "If you wish to overwrite the existing data, set overwrite to True")

    return existing_df

In [ ]:
def get_simulation_information(save_location, env, vessels, overwrite = False):
    """
    Create a dataframe from all equipment logs
    """

    object_dict = {"SimulationID": [], 
                   "VesselID": [], 
                   "Energy": [], 
                   "Energy stationary": [], 
                   "Time": [], 
                   "Waiting": [], 
                   "Path": []}
    
    try:
        unique_df = pd.read_csv(save_location + "simulation_results.csv")
    except FileNotFoundError:
        unique_df = pd.DataFrame.from_dict(object_dict)

    for vessel in vessels:
        object_dict["SimulationID"].append(env.id)
        object_dict["VesselID"].append(vessel.id)
        object_dict["Path"].append(vessel.complete_path.name.replace("Path_", ""))
        
        duration = (vessel.log["Timestamp"][-1] - vessel.log["Timestamp"][0]) / 3600
        object_dict["Time"].append(duration)
        
        waiting = 0
        for i, log in enumerate(vessel.log["Message"]):
            log_clean = log.replace(" start", "")
            log_clean = log_clean.replace(" stop", "")
            
            if log_clean in ["waiting to pass bridge", "waiting to pass lock", "waiting to pass edge"]:
                waiting_start = vessel.log["Timestamp"][i]
                
                for j, log_stop in enumerate(vessel.log["Message"][i::]):
                    if log_stop.replace(" stop", "") == log_clean:
                        waiting_stop = vessel.log["Timestamp"][i + j]
                        waiting += waiting_stop - waiting_start
                        break
        
        object_dict["Waiting"].append(waiting / 3600)
        
        vessel.calculate_energy_consumption()
        object_dict["Energy"].append(vessel.energy_use["total_energy"])
        object_dict["Energy stationary"].append(vessel.energy_use["stationary"])

    # Create durations column
    object_df = pd.DataFrame.from_dict(object_dict)

    # Check if combination of simulation ID and object ID already exists
    if len(unique_df["SimulationID"]) == 0:
        unique_df = object_df

    elif not (unique_df["SimulationID"] == env.id).any():
        unique_df = pd.concat([unique_df, object_df], ignore_index = True)

    elif overwrite == True:
        drop_rows = []

        for i, row in enumerate(unique_df["SimulationID"] == env.id):
            if row == True:
                drop_rows.append(i)

        unique_df = unique_df.drop(drop_rows, axis = 0)
        unique_df = pd.concat([unique_df, object_df], ignore_index = True)

    else:
        raise KeyError("Simulation ID or simulation name already exist. " + 
                       "If you wish to overwrite the existing data, set overwrite to True")

    return unique_df

#### Save simulation

In [ ]:
# Simulation information
env.name = "BathseLock - NS - High"
env.id = "2030bc86-4a3d-11e9-ba19-b469212bff5b"

unique_simulations = append_dataframe(save_location, env, overwrite = True)
unique_simulations.to_csv(save_location + "simulations.csv", index = False)

#### Save vessel information

In [ ]:
unique_simulation_results = get_simulation_information(save_location, env, vessels, overwrite = True)
unique_simulation_results.to_csv(save_location + "simulation_results.csv", index = False)